In [36]:
!pip install -U kaleido

# Equity Research Report Generator

A comprehensive fundamental analysis tool for generating institutional-quality equity research reports.

## Features
- Company overview and financial data collection (via yfinance)
- Comprehensive financial ratio analysis
- Peer comparison and relative valuation
- Sentiment and momentum analysis
- Interactive visualizations
- Automated scoring and recommendation system

---

## Configuration

**Change the ticker below to analyze any stock:**

In [1]:
%cd /content

/content


In [2]:
!git clone https://github.com/matthudson1223/geyser.git

fatal: destination path 'geyser' already exists and is not an empty directory.


In [3]:
%cd /content/geyser

/content/geyser


In [4]:
# =============================================================================
# CONFIGURATION - CHANGE THIS TO ANALYZE A DIFFERENT STOCK
# =============================================================================

TICKER = "NVDA"  # Change this to any valid stock ticker

# Examples:
# TICKER = "AAPL"   # Apple
# TICKER = "MSFT"   # Microsoft
# TICKER = "GOOGL"  # Alphabet
# TICKER = "AMZN"   # Amazon
# TICKER = "META"   # Meta
# TICKER = "TSLA"   # Tesla
# TICKER = "JPM"    # JPMorgan
# TICKER = "JNJ"    # Johnson & Johnson

print(f"Analyzing: {TICKER}")

Analyzing: NVDA


## Setup

In [5]:
# Import required libraries
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from src.data_collector import DataCollector
from src.financial_analysis import FinancialAnalyzer
from src.peer_comparison import PeerComparison
from src.sentiment_analysis import SentimentAnalyzer
from src.visualizations import ChartGenerator
from src.report_generator import ReportGenerator, ScoringEngine

# For interactive charts
import plotly.io as pio
pio.renderers.default = 'notebook'

print("Setup complete!")

Setup complete!


---
## Phase 1: Data Collection

In [6]:
# Collect all data for the ticker
print(f"Collecting data for {TICKER}...")
collector = DataCollector(TICKER)
data = collector.get_all_data()

# Display company overview
overview = data['overview']
print(f"\n{'='*60}")
print(f"Company: {overview['name']}")
print(f"Sector: {overview['sector']}")
print(f"Industry: {overview['industry']}")
print(f"Market Cap: ${overview['market_cap']:,.0f}" if overview['market_cap'] else "Market Cap: N/A")
print(f"Current Price: ${overview['current_price']:.2f}" if overview['current_price'] else "Price: N/A")
print(f"{'='*60}")

Data collection complete for NVDA

Company: NVIDIA Corporation
Sector: Technology
Industry: Semiconductors
Market Cap: $4,565,428,076,544
Current Price: $187.51


In [7]:
# Display business description
print("Business Summary:")
print("-" * 60)
description = overview.get('description', 'N/A')
# Wrap text for better display
import textwrap
print(textwrap.fill(description[:1000] + "..." if len(description) > 1000 else description, width=80))

Business Summary:
------------------------------------------------------------
NVIDIA Corporation, a computing infrastructure company, provides graphics and
compute and networking solutions in the United States, Singapore, Taiwan, China,
Hong Kong, and internationally. The Compute & Networking segment includes its
Data Centre accelerated computing platforms and artificial intelligence
solutions and software; networking; automotive platforms and autonomous and
electric vehicle solutions; Jetson for robotics and other embedded platforms;
and DGX Cloud computing services. The Graphics segment offers GeForce GPUs for
gaming and PCs, the GeForce NOW game streaming service and related
infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for
enterprise workstation graphics; virtual GPU or vGPU software for cloud-based
visual and virtual computing; automotive platforms for infotainment systems; and
Omniverse software for building and operating industrial AI and digital tw

In [8]:
# Display key statistics
stats = data['key_statistics']
print("\nKey Statistics:")
print("-" * 40)

key_stats = [
    ('P/E (Trailing)', stats.get('pe_trailing')),
    ('P/E (Forward)', stats.get('pe_forward')),
    ('PEG Ratio', stats.get('peg_ratio')),
    ('Price/Book', stats.get('price_to_book')),
    ('EV/EBITDA', stats.get('ev_to_ebitda')),
    ('Profit Margin', stats.get('profit_margin')),
    ('ROE', stats.get('roe')),
    ('Beta', stats.get('beta')),
]

for name, value in key_stats:
    if value is not None:
        if 'Margin' in name or 'ROE' in name:
            print(f"{name}: {value:.1%}")
        else:
            print(f"{name}: {value:.2f}")


Key Statistics:
----------------------------------------
P/E (Trailing): 46.41
P/E (Forward): 24.82
Price/Book: 38.33
EV/EBITDA: 40.64
Profit Margin: 53.0%
ROE: 107.4%
Beta: 2.28


---
## Phase 2: Financial Analysis

In [9]:
# Perform comprehensive financial analysis
analyzer = FinancialAnalyzer(data)
analysis = analyzer.get_comprehensive_analysis()
analysis['historical_prices'] = data.get('historical_prices')

print("Financial Analysis Complete!")

Financial Analysis Complete!


In [10]:
# Display valuation metrics
from IPython.display import display, HTML
import pandas as pd

val = analysis['valuation']
valuation_data = {
    'Metric': ['P/E (TTM)', 'P/E (Forward)', 'PEG Ratio', 'P/B', 'P/S', 'EV/EBITDA', 'EV/Revenue', 'P/FCF'],
    'Value': [
        f"{val.get('pe_trailing'):.2f}" if val.get('pe_trailing') else 'N/A',
        f"{val.get('pe_forward'):.2f}" if val.get('pe_forward') else 'N/A',
        f"{val.get('peg_ratio'):.2f}" if val.get('peg_ratio') else 'N/A',
        f"{val.get('price_to_book'):.2f}" if val.get('price_to_book') else 'N/A',
        f"{val.get('price_to_sales'):.2f}" if val.get('price_to_sales') else 'N/A',
        f"{val.get('ev_to_ebitda'):.2f}" if val.get('ev_to_ebitda') else 'N/A',
        f"{val.get('ev_to_revenue'):.2f}" if val.get('ev_to_revenue') else 'N/A',
        f"{val.get('price_to_fcf'):.2f}" if val.get('price_to_fcf') else 'N/A',
    ]
}

print("Valuation Metrics:")
display(pd.DataFrame(valuation_data))

Valuation Metrics:


,Metric,Value
0,P/E (TTM),46.41
1,P/E (Forward),24.82
2,PEG Ratio,N/A
3,P/B,38.33
4,P/S,24.40
5,EV/EBITDA,40.64
6,EV/Revenue,24.47
7,P/FCF,85.68


In [11]:
# Display profitability metrics
prof = analysis['profitability']

profitability_data = {
    'Metric': ['Gross Margin', 'Operating Margin', 'Net Margin', 'ROE', 'ROA', 'ROIC'],
    'Value': [
        f"{prof.get('gross_margin'):.1%}" if prof.get('gross_margin') else 'N/A',
        f"{prof.get('operating_margin'):.1%}" if prof.get('operating_margin') else 'N/A',
        f"{prof.get('net_margin'):.1%}" if prof.get('net_margin') else 'N/A',
        f"{prof.get('roe'):.1%}" if prof.get('roe') else 'N/A',
        f"{prof.get('roa'):.1%}" if prof.get('roa') else 'N/A',
        f"{prof.get('roic'):.1%}" if prof.get('roic') else 'N/A',
    ]
}

print("Profitability Metrics:")
display(pd.DataFrame(profitability_data))

Profitability Metrics:


,Metric,Value
0,Gross Margin,70.0%
1,Operating Margin,63.2%
2,Net Margin,53.0%
3,ROE,107.4%
4,ROA,53.5%
5,ROIC,87.2%


In [12]:
# Display growth metrics
growth = analysis['growth']

growth_data = {
    'Metric': ['Revenue Growth (YoY)', 'Revenue CAGR (3Y)', 'EPS Growth (YoY)', 'FCF Growth (YoY)'],
    'Value': [
        f"{growth.get('revenue_growth_1y'):.1%}" if growth.get('revenue_growth_1y') else 'N/A',
        f"{growth.get('revenue_cagr_3y'):.1%}" if growth.get('revenue_cagr_3y') else 'N/A',
        f"{growth.get('eps_growth_1y'):.1%}" if growth.get('eps_growth_1y') else 'N/A',
        f"{growth.get('fcf_growth_1y'):.1%}" if growth.get('fcf_growth_1y') else 'N/A',
    ]
}

print("Growth Metrics:")
display(pd.DataFrame(growth_data))

Growth Metrics:


,Metric,Value
0,Revenue Growth (YoY),114.2%
1,Revenue CAGR (3Y),69.3%
2,EPS Growth (YoY),145.5%
3,FCF Growth (YoY),125.2%


---
## Phase 3: Peer Comparison

In [13]:
# Perform peer comparison
peer_comp = PeerComparison(TICKER, data)
peers = peer_comp.identify_peers()
print(f"Identified peers: {', '.join(peers)}")

print("\nCollecting peer data...")
peer_comp.collect_peer_data()
peer_summary = peer_comp.get_peer_summary()
print("Peer comparison complete!")

Identified peers: AMD, INTC, QCOM, AVGO, TXN


Peers: AMD, INTC, QCOM, AVGO, TXN
  Fetching data for AMD...
  Fetching data for INTC...
  Fetching data for QCOM...
  Fetching data for AVGO...
  Fetching data for TXN...
Collected data for 5 peers
Peer comparison complete!


In [14]:
# Display peer comparison table
print("\nPeer Comparison Matrix:")
display(peer_summary['formatted_comparison'])


Peer Comparison Matrix:


,NVDA,AMD,INTC,QCOM,AVGO,TXN,Peer Avg
Metric,,,,,,,
P/E (TTM),46.41,112.37,604.50,34.44,73.63,32.02,171.39
P/E (Forward),24.82,33.23,60.91,13.91,24.98,28.43,32.29
PEG Ratio,N/A,N/A,N/A,N/A,N/A,N/A,N/A
P/B,38.33,5.75,1.62,8.76,5.91,9.59,6.33
P/S,24.40,10.91,3.24,4.21,25.96,9.24,10.71
EV/EBITDA,40.64,57.26,19.03,13.77,4.97,21.70,23.35
EV/Revenue,24.47,10.82,3.72,4.35,2.73,9.82,6.29
Gross Margin,70.0%,51.5%,33.0%,55.4%,77.3%,57.5%,54.9%
Operating Margin,63.2%,13.7%,6.3%,26.2%,31.8%,36.7%,23.0%


In [15]:
# Display relative valuation assessment
rel_val = peer_summary['relative_valuation']
justification = peer_summary['valuation_justification']

print("Relative Valuation Assessment:")
print("=" * 50)
print(f"Status: {rel_val.get('assessment', 'N/A')}")
print(f"Detail: {rel_val.get('assessment_detail', 'N/A')}")
print(f"\nConclusion: {justification.get('conclusion', 'N/A')}")

if justification.get('factors_supporting'):
    print("\nFactors Supporting:")
    for f in justification['factors_supporting']:
        print(f"  + {f}")

if justification.get('factors_against'):
    print("\nFactors Against:")
    for f in justification['factors_against']:
        print(f"  - {f}")

Relative Valuation Assessment:
Status: Significant Premium
Detail: Trading at 122% premium to peers on average

Conclusion: Premium appears JUSTIFIED by superior fundamentals

Factors Supporting:
  + Higher revenue growth (62.5% vs peer avg 15.8%)
  + Higher net margin (53.0% vs peer avg 17.7%)
  + Higher roe (107.4% vs peer avg 17.9%)
  + Higher roa (53.5% vs peer avg 7.5%)


---
## Phase 4: Sentiment Analysis

In [16]:
# Perform sentiment analysis
sentiment_analyzer = SentimentAnalyzer(TICKER, data)
sentiment = sentiment_analyzer.get_sentiment_summary()

print("Sentiment Analysis Complete!")

Sentiment Analysis Complete!


In [17]:
# Display analyst recommendations
analyst = sentiment['analyst_recommendations']

print("Analyst Recommendations:")
print("=" * 50)
print(f"Consensus: {analyst.get('consensus', 'N/A')}")
print(f"Total Analysts: {analyst.get('total_analysts', 'N/A')}")
print(f"Mean Rating: {analyst.get('mean_rating', 'N/A'):.2f}" if analyst.get('mean_rating') else "Mean Rating: N/A")

if analyst.get('recommendation_breakdown'):
    print("\nBreakdown:")
    for rating, count in analyst['recommendation_breakdown'].items():
        print(f"  {rating}: {count}")

pt = analyst.get('price_targets', {})
if pt.get('target_mean'):
    print(f"\nPrice Targets:")
    print(f"  Current: ${pt.get('current_price', 0):.2f}")
    print(f"  Mean Target: ${pt.get('target_mean', 0):.2f}")
    print(f"  Range: ${pt.get('target_low', 0):.2f} - ${pt.get('target_high', 0):.2f}")
    if pt.get('upside_pct'):
        print(f"  Implied Upside: {pt['upside_pct']:.1f}%")

Analyst Recommendations:
Consensus: Buy
Total Analysts: 64
Mean Rating: 1.91

Breakdown:
  Strong Buy: 11
  Buy: 49
  Hold: 3
  Sell: 1
  Strong Sell: 0

Price Targets:
  Current: $187.51
  Mean Target: $253.02
  Range: $140.00 - $352.00
  Implied Upside: 34.9%


In [18]:
# Display momentum analysis
momentum = sentiment['momentum']

print("Technical Momentum:")
print("=" * 50)
print(f"Overall: {momentum.get('overall_momentum', 'N/A')}")
print(f"Score: {momentum.get('momentum_score', 'N/A')}/10")

print("\nSignals:")
for signal in momentum.get('technical_signals', []):
    print(f"  - {signal}")

Technical Momentum:
Overall: Strong Bullish
Score: 7.0/10

Signals:
  - Positive 1M momentum (+5%+)
  - Price above 50-day MA
  - Price above 200-day MA
  - Golden cross (50MA > 200MA)


In [19]:
# Overall sentiment
print("\n" + "=" * 50)
print(f"OVERALL SENTIMENT: {sentiment.get('overall_sentiment', 'N/A')}")
print(f"SENTIMENT SCORE: {sentiment.get('overall_sentiment_score', 'N/A')}/10")
print("=" * 50)


OVERALL SENTIMENT: Very Bullish
SENTIMENT SCORE: 7.6/10


---
## Phase 5: Visualizations

In [20]:
# Initialize chart generator
chart_gen = ChartGenerator(TICKER, 'output/charts')

In [37]:
# Price Chart with Moving Averages
price_chart = chart_gen.create_price_chart(data['historical_prices'], years=2)
if price_chart:
    price_chart.show()

In [22]:
# Financial Trends Chart
trends_chart = chart_gen.create_financial_trends_chart(analysis['growth'])
if trends_chart:
    trends_chart.show()

In [23]:
# Margin Analysis Chart
margin_chart = chart_gen.create_margin_analysis_chart(analysis['profitability'])
if margin_chart:
    margin_chart.show()

In [24]:
# Peer Valuation Comparison
peer_chart = chart_gen.create_peer_valuation_chart(peer_summary['comparison_matrix'])
if peer_chart:
    peer_chart.show()

In [25]:
# Growth vs Valuation Scatter
scatter_chart = chart_gen.create_growth_vs_valuation_scatter(peer_summary['comparison_matrix'])
if scatter_chart:
    scatter_chart.show()

In [26]:
# Sentiment Gauge
sentiment_gauge = chart_gen.create_sentiment_gauge(
    sentiment.get('overall_sentiment_score', 5),
    sentiment.get('overall_sentiment', 'Neutral')
)
if sentiment_gauge:
    sentiment_gauge.show()

---
## Phase 6: Investment Recommendation

In [27]:
# Generate report and scores
report_gen = ReportGenerator(TICKER, data, analysis, peer_summary, sentiment)
scores = report_gen.get_scores()

print("Scoring Analysis Complete!")

Scoring Analysis Complete!


In [28]:
# Display scoring matrix
print("Investment Scoring Matrix:")
print("=" * 60)

scoring_data = []
for cat, cat_scores in scores['categories'].items():
    scoring_data.append({
        'Category': cat,
        'Weight': f"{cat_scores['weight']:.0%}",
        'Score': f"{cat_scores['score']:.1f}",
        'Weighted': f"{cat_scores['weighted']:.2f}"
    })

scoring_df = pd.DataFrame(scoring_data)
display(scoring_df)

print(f"\nTOTAL SCORE: {scores['total_score']}/10")
print(f"RECOMMENDATION: {scores['recommendation']}")

Investment Scoring Matrix:


,Category,Weight,Score,Weighted
0,Valuation,25%,3.5,0.88
1,Growth,20%,9.5,1.90
2,Profitability,20%,9.5,1.90
3,Financial Health,15%,8.5,1.27
4,Momentum/Sentiment,10%,9.0,0.90
5,Quality/Moat,10%,8.5,0.85



TOTAL SCORE: 7.7/10
RECOMMENDATION: Buy


In [29]:
# Display score breakdown chart
score_chart = chart_gen.create_recommendation_score_chart(scores)
if score_chart:
    score_chart.show()

In [30]:
# Display scoring factors
print("\nScoring Factors by Category:")
print("=" * 60)

for category, factors in scores['all_factors'].items():
    if factors:
        print(f"\n{category.upper()}:")
        for factor in factors:
            print(f"  - {factor}")


Scoring Factors by Category:

VALUATION:
  - Elevated P/E (46.4x)
  - Forward P/E significantly lower - growth expected
  - Significant premium (122%) vs peers
  - Low FCF yield (1.2%)

GROWTH:
  - Excellent revenue growth (114.2%)
  - Strong 3-year revenue CAGR (69.3%)
  - Strong EPS growth (145.5%)
  - Strong FCF growth (125.2%)

PROFITABILITY:
  - Excellent gross margin (70.0%)
  - Strong operating margin (63.2%)
  - Excellent ROE (107.4%)
  - High ROIC (87.2%) - efficient capital allocation

HEALTH:
  - Strong current ratio (4.47)
  - Low leverage (D/E: 9%)
  - Excellent interest coverage (329.8x)
  - Strong Altman Z-Score (89.97) - low bankruptcy risk

MOMENTUM:
  - Positive 1M momentum (+5%+)
  - Price above 50-day MA
  - Price above 200-day MA
  - Strong analyst buy consensus
  - Significant upside to price target (35%)
  - Strong earnings beat rate (100%)

QUALITY:
  - High margins suggest pricing power/moat
  - High ROE & ROIC suggest durable competitive advantage
  - Consist

---
## Bull and Bear Cases

In [31]:
from src.report_generator import ThesisGenerator

thesis_gen = ThesisGenerator(analysis, peer_summary, sentiment, scores)

bull_case = thesis_gen.generate_bull_case()
bear_case = thesis_gen.generate_bear_case()
key_metrics = thesis_gen.generate_key_metrics_to_monitor()

print("BULL CASE:")
print("=" * 50)
for i, point in enumerate(bull_case, 1):
    print(f"{i}. {point}")

print("\nBEAR CASE:")
print("=" * 50)
for i, point in enumerate(bear_case, 1):
    print(f"{i}. {point}")

print("\nKEY METRICS TO MONITOR:")
print("=" * 50)
for metric in key_metrics:
    print(f"- {metric}")

BULL CASE:
1. Forward P/E significantly lower - growth expected
2. Excellent revenue growth (114.2%)
3. Strong 3-year revenue CAGR (69.3%)
4. Strong EPS growth (145.5%)
5. Strong FCF growth (125.2%)

BEAR CASE:
1. Elevated P/E (46.4x)
2. Forward P/E significantly lower - growth expected
3. Low FCF yield (1.2%)
4. Low leverage (D/E: 9%)
5. Strong Altman Z-Score (89.97) - low bankruptcy risk

KEY METRICS TO MONITOR:
- Quarterly revenue growth and guidance
- Operating margin trends
- Free cash flow generation
- Data center/AI segment growth
- Gross margin trends by segment
- Earnings growth to justify premium valuation


---
## Generate Full Report

In [33]:
# Generate and save the full report
import os
os.makedirs('output', exist_ok=True)

report = report_gen.generate_full_report()

report_path = f"output/{TICKER}_research_report.md"
with open(report_path, 'w') as f:
    f.write(report)

print(f"Full report saved to: {report_path}")
print(f"\nReport length: {len(report):,} characters")

Full report saved to: output/NVDA_research_report.md

Report length: 9,579 characters


In [34]:
# Display the report
from IPython.display import Markdown
display(Markdown(report))


# NVDA Equity Research Report
## Executive Summary

**Company:** NVIDIA Corporation
**Sector:** Technology | **Industry:** Semiconductors

**Investment Recommendation: Buy** (Score: 7.7/10)

| Metric           | Value   |
|:-----------------|:--------|
| Current Price    | $188    |
| Market Cap       | $4.57T  |
| P/E (TTM)        | 46.41x  |
| Revenue Growth   | 114.2%  |
| Net Margin       | 53.0%   |
| Investment Score | 7.7/10  |
| Recommendation   | Buy     |

*Report generated on December 29, 2025*

---

## Company Overview

### Business Description

NVIDIA Corporation, a computing infrastructure company, provides graphics and compute and networking solutions in the United States, Singapore, Taiwan, China, Hong Kong, and internationally. The Compute & Networking segment includes its Data Centre accelerated computing platforms and artificial intelligence solutions and software; networking; automotive platforms and autonomous and electric vehicle solutions; Jetson for robotics and other embedded platforms; and DGX Cloud computing services. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building and operating industrial AI and digital twin applications. It also customized agentic solutions designed in collaboration with NVIDIA to accelerate enterprise AI adoption. The company's products are used in gaming, professional visualization, data center, and automotive markets. It sells its products to original equipment manufacturers, original device manufacturers, system integrators and distributors, independent software vendors, cloud service providers, consumer internet companies, add-in board manufacturers, distributors, automotive manufacturers and tier-1 automotive suppliers, and other ecosystem participants. NVIDIA Corporation was incorporated in 1993 and is headquartered in Santa Clara, California.

### Key Facts

| Attribute | Value |
|-----------|-------|
| Headquarters | United States |
| Employees | 36,000 |
| Website | https://www.nvidia.com |
| Market Cap | $4.57T |
| Enterprise Value | $4.58T |

### Price Performance

| Metric | Value |
|--------|-------|
| 52-Week High | $212 |
| 52-Week Low | $87 |
| % From 52W High | -1162.9% |
| 50-Day MA | $186 |
| 200-Day MA | $159 |

---

## Financial Analysis

### Valuation Metrics

| Metric | Value | Assessment |
|--------|-------|------------|
| P/E (TTM) | 46.41x | Premium |
| P/E (Forward) | 24.82x | - |
| PEG Ratio | N/A | Expensive |
| P/B | 38.33x | - |
| P/S | 24.40x | - |
| EV/EBITDA | 40.64x | - |
| P/FCF | 85.68x | - |

### Profitability Metrics

| Metric | Value |
|--------|-------|
| Gross Margin | 70.0% |
| Operating Margin | 63.2% |
| Net Margin | 53.0% |
| ROE | 107.4% |
| ROA | 53.5% |
| ROIC | 87.2% |

### Growth Metrics

| Metric | Value |
|--------|-------|
| Revenue Growth (YoY) | 114.2% |
| Revenue CAGR (3Y) | 69.3% |
| EPS Growth (YoY) | 145.5% |
| FCF Growth (YoY) | 125.2% |

### Financial Health

| Metric | Value | Assessment |
|--------|-------|------------|
| Current Ratio | 4.47x | Strong |
| Debt/Equity | 0.09x | Low |
| Interest Coverage | 329.77x | Strong |
| Altman Z-Score | 89.97 | Safe |

---

## Peer Comparison

### Peers Analyzed
AMD, INTC, QCOM, AVGO, TXN

### Comparative Metrics

| Metric           | NVDA     | AMD     | INTC    | QCOM    | AVGO     | TXN     | Peer Avg   |
|:-----------------|:---------|:--------|:--------|:--------|:---------|:--------|:-----------|
| P/E (TTM)        | 46.41    | 112.37  | 604.50  | 34.44   | 73.63    | 32.02   | 171.39     |
| P/E (Forward)    | 24.82    | 33.23   | 60.91   | 13.91   | 24.98    | 28.43   | 32.29      |
| PEG Ratio        | N/A      | N/A     | N/A     | N/A     | N/A      | N/A     | N/A        |
| P/B              | 38.33    | 5.75    | 1.62    | 8.76    | 5.91     | 9.59    | 6.33       |
| P/S              | 24.40    | 10.91   | 3.24    | 4.21    | 25.96    | 9.24    | 10.71      |
| EV/EBITDA        | 40.64    | 57.26   | 19.03   | 13.77   | 4.97     | 21.70   | 23.35      |
| EV/Revenue       | 24.47    | 10.82   | 3.72    | 4.35    | 2.73     | 9.82    | 6.29       |
| Gross Margin     | 70.0%    | 51.5%   | 33.0%   | 55.4%   | 77.3%    | 57.5%   | 54.9%      |
| Operating Margin | 63.2%    | 13.7%   | 6.3%    | 26.2%   | 31.8%    | 36.7%   | 23.0%      |
| Net Margin       | 53.0%    | 10.3%   | 0.4%    | 12.5%   | 36.2%    | 29.2%   | 17.7%      |
| ROE              | 107.4%   | 5.3%    | 0.2%    | 23.3%   | 31.0%    | 29.8%   | 17.9%      |
| ROA              | 53.5%    | 2.6%    | -0.5%   | 14.7%   | 9.8%     | 10.7%   | 7.5%       |
| Revenue Growth   | 62.5%    | 35.6%   | 2.8%    | 10.0%   | 16.4%    | 14.2%   | 15.8%      |
| Earnings Growth  | 66.7%    | 60.3%   | N/A     | N/A     | 188.1%   | 0.7%    | 83.0%      |
| Current Ratio    | 4.47     | 2.31    | 1.60    | 2.82    | 1.71     | 4.45    | 2.58       |
| Debt/Equity      | 9.10     | 6.37    | 39.88   | 73.77   | 166.03   | 84.48   | 74.10      |
| Market Cap ($B)  | $4565.4B | $349.4B | $173.0B | $186.5B | $1658.2B | $159.5B | $505.3B    |
| Beta             | 2.28     | 1.93    | 1.34    | 1.21    | 1.20     | 0.99    | 1.34       |
| 1Y Return        | 34.0%    | 71.6%   | 77.4%   | 11.5%   | 43.8%    | -5.5%   | 39.8%      |
| YTD Return       | 35.6%    | 77.9%   | 79.4%   | 15.0%   | 52.1%    | -3.1%   | 44.3%      |

### Relative Valuation Assessment

**Status:** Significant Premium
**Detail:** Trading at 122% premium to peers on average

### Valuation Justification Analysis

**Conclusion:** Premium appears JUSTIFIED by superior fundamentals

**Factors Supporting Valuation:**

- Higher revenue growth (62.5% vs peer avg 15.8%)
- Higher net margin (53.0% vs peer avg 17.7%)
- Higher roe (107.4% vs peer avg 17.9%)
- Higher roa (53.5% vs peer avg 7.5%)

**Factors Against Valuation:**

---

## Sentiment & Catalyst Analysis

### Analyst Recommendations

| Metric | Value |
|--------|-------|
| Consensus | Buy |
| Total Analysts | 64 |
| Mean Rating | 1.91 (1=Strong Buy, 5=Strong Sell) |

### Price Targets

| Metric | Value |
|--------|-------|
| Current Price | $188 |
| Mean Target | $253 |
| Low Target | $140 |
| High Target | $352 |
| Implied Upside | 3493.2% |

### Earnings Performance

| Metric | Value |
|--------|-------|
| Quarters Analyzed | 4 |
| Beat Rate | 100.0% |
| Avg Surprise | 5.2% |
| Trend | Improving |

### Technical Momentum

**Overall Assessment:** Strong Bullish
**Momentum Score:** 7.0/10

**Signals:**

- Positive 1M momentum (+5%+)
- Price above 50-day MA
- Price above 200-day MA
- Golden cross (50MA > 200MA)

### Overall Sentiment Score: 7.6/10 (Very Bullish)

---

## Investment Thesis

### Bull Case

1. Forward P/E significantly lower - growth expected
2. Excellent revenue growth (114.2%)
3. Strong 3-year revenue CAGR (69.3%)
4. Strong EPS growth (145.5%)
5. Strong FCF growth (125.2%)

### Bear Case

1. Elevated P/E (46.4x)
2. Forward P/E significantly lower - growth expected
3. Low FCF yield (1.2%)
4. Low leverage (D/E: 9%)
5. Strong Altman Z-Score (89.97) - low bankruptcy risk

### Key Metrics to Monitor

- Quarterly revenue growth and guidance
- Operating margin trends
- Free cash flow generation
- Data center/AI segment growth
- Gross margin trends by segment
- Earnings growth to justify premium valuation

---

## Valuation & Recommendation

### Quantitative Scoring Matrix

| Category           | Weight   | Score   | Weighted   |
|:-------------------|:---------|:--------|:-----------|
| Valuation          | 25%      | 3.5     | 0.88       |
| Growth             | 20%      | 9.5     | 1.90       |
| Profitability      | 20%      | 9.5     | 1.90       |
| Financial Health   | 15%      | 8.5     | 1.27       |
| Momentum/Sentiment | 10%      | 9.0     | 0.90       |
| Quality/Moat       | 10%      | 8.5     | 0.85       |
| **TOTAL**          | 100%     |         | **7.7**    |

### Final Recommendation: **Buy**

#### Score Interpretation:
- **Strong Buy** (≥8.0): Compelling risk/reward, high conviction
- **Buy** (6.5-7.9): Favorable outlook, attractive entry point
- **Hold** (5.0-6.4): Fairly valued, maintain position
- **Sell** (3.5-4.9): Unfavorable risk/reward
- **Strong Sell** (<3.5): Significant downside risk

### Investment Commentary

NVDA presents a compelling investment opportunity with strong fundamentals
across multiple dimensions. The company demonstrates notable strengths in growth, profitability, financial health, momentum/sentiment, quality/moat. However, concerns remain around valuation. 

**Ideal Investor Profile:** Growth-oriented investors with medium to high risk tolerance.

**Position Sizing Consideration:** Moderate conviction - consider standard position size (1-3%).

---

## Disclaimer

*This report is for informational purposes only and does not constitute financial advice,
investment recommendations, or an offer to buy or sell any securities. The analysis is based
on publicly available data from Yahoo Finance and may contain errors or inaccuracies.
Past performance is not indicative of future results. Always conduct your own research and
consult with a qualified financial advisor before making investment decisions.*

*Data source: Yahoo Finance (yfinance)*
*Report generated using automated analysis tools*


---
## Save Charts

In [38]:
# Save all charts
import os
os.makedirs('output/charts', exist_ok=True)

charts = chart_gen.save_all_charts(
    analysis=analysis,
    peer_comparison=peer_summary['comparison_matrix'],
    sentiment=sentiment,
    scores=scores,
    format='html'
)

print(f"Saved {len(charts)} charts:")
for name, path in charts.items():
    print(f"  - {name}: {path}")

Saved 7 charts:
  - price_chart: output/charts/NVDA_price_chart.html
  - financial_trends: output/charts/NVDA_financial_trends.html
  - margin_analysis: output/charts/NVDA_margin_analysis.html
  - peer_valuation: output/charts/NVDA_peer_valuation.html
  - growth_vs_valuation: output/charts/NVDA_growth_vs_valuation.html
  - sentiment_gauge: output/charts/NVDA_sentiment_gauge.html
  - score_breakdown: output/charts/NVDA_score_breakdown.html


---
## Summary

In [36]:
print("\n" + "=" * 60)
print(f"  ANALYSIS COMPLETE: {TICKER}")
print("=" * 60)
print(f"\n  Company: {analysis.get('company_name', 'N/A')}")
print(f"  Sector: {analysis.get('sector', 'N/A')}")
print(f"  Industry: {analysis.get('industry', 'N/A')}")
print(f"\n  Current Price: ${overview.get('current_price', 0):.2f}")
print(f"  Market Cap: ${overview.get('market_cap', 0)/1e9:.1f}B")
print(f"\n  Investment Score: {scores['total_score']}/10")
print(f"  Recommendation: {scores['recommendation']}")
print(f"\n  Report: {report_path}")
print("\n" + "=" * 60)


  ANALYSIS COMPLETE: NVDA

  Company: NVIDIA Corporation
  Sector: Technology
  Industry: Semiconductors

  Current Price: $187.51
  Market Cap: $4565.4B

  Investment Score: 7.7/10
  Recommendation: Buy

  Report: output/NVDA_research_report.md



---

## Disclaimer

*This report is for informational purposes only and does not constitute financial advice. Always conduct your own research and consult with a qualified financial advisor before making investment decisions.*